# Setup

In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
#headers = ['Date', 'Time', 'Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
dictionary_for_needed_type = {
    'Global_active_power': 'float64',
    'Global_reactive_power': 'float64',
    'Voltage': 'float64',
    'Global_intensity': 'float64',
    'Sub_metering_1': 'float64',
    'Sub_metering_2': 'float64',
    'Sub_metering_3': 'float64',
                             }

In [ ]:
start_time = datetime.datetime.now()

df = pd.read_csv("household_power_consumption.txt", sep=";", low_memory=False)
#, index_col=["Date", "Time"]

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

In [ ]:
start_time = datetime.datetime.now()

df = df.drop(df.loc[df["Date"]=='?'].index)
df = df.drop(df.loc[df["Time"]=='?'].index)
df = df.drop(df.loc[df["Global_active_power"]=='?'].index)
df = df.drop(df.loc[df["Global_reactive_power"]=='?'].index)
df = df.drop(df.loc[df["Voltage"]=='?'].index)
df = df.drop(df.loc[df["Global_intensity"]=='?'].index)
df = df.drop(df.loc[df["Sub_metering_1"]=='?'].index)
df = df.drop(df.loc[df["Sub_metering_2"]=='?'].index)
df = df.drop(df.loc[df["Sub_metering_3"]=='?'].index)

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

In [ ]:
start_time = datetime.datetime.now()

df = df.astype(dictionary_for_needed_type)

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

# Завдання першого рівня
Виконати всі завдання, використовуючи як numpy array, так і dataframe, проаналізувати часові витрати на виконання процедур (профілювання часу виконання), зробити висновки щодо ситуацій, в яких має сенс віддати перевагу тій чи іншій структурі даних. Висновки оформити звітом із зазначеним часом виконання та оцінкою по 5-бальній шкалі зручності виконання операцій відбору).

## 1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.

In [ ]:
start_time = datetime.datetime.now()

print(df[df["Global_active_power"]>5][["Date", "Time", 'Global_active_power', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

## 2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.

In [ ]:
start_time = datetime.datetime.now()

print(df[df["Voltage"]>235][["Date", "Time", 'Voltage', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

## 3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.

In [ ]:
start_time = datetime.datetime.now()

df19_20 = df[(df["Global_intensity"] >= 19) & (df["Global_intensity"] <= 20)][["Date", "Time", 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
df19_20
print(df19_20[df19_20["Sub_metering_2"] > df19_20["Sub_metering_3"]])


end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

## 4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії, а також

In [ ]:
start_time = datetime.datetime.now()

random_df = df.sample(500000)
random_df['Average_of_3'] = (random_df['Sub_metering_1']+random_df['Sub_metering_2']+random_df['Sub_metering_3'])/3
print(random_df[["Date", "Time", 'Average_of_3', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")

## 5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [ ]:
# start_time = np.datetime64('now', 'ms')


fifth_df = df[(df['Time']>= "18:30:00") & (df['Global_active_power']>6) & (df["Sub_metering_2"]>df["Sub_metering_1"]) & (df["Sub_metering_2"]>df["Sub_metering_3"])]
# fifth_df
order = []
for i in range(0, len(fifth_df)//2, 3):
    order.append(fifth_df.iloc[i])
for i in range(len(fifth_df)//2, len(fifth_df), 4):
    order.append(fifth_df.iloc[i])
# end_time = np.datetime64('now', 'ms')
df_fifth = pd.DataFrame.from_records(order)
print(df_fifth)

end_time = datetime.datetime.now()
execution_time = end_time - start_time
print(f"{execution_time.total_seconds() * 1000} miliseconds or {execution_time.total_seconds()} seconds")